# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
display(data.head(10)) # получение таблицы
data.info() # получение общей информации о данных в таблице
data['children'].value_counts()
data['dob_years'].value_counts()
data['education'].value_counts()
data['family_status'].value_counts()
data['gender'].value_counts()
data['income_type'].value_counts()
data['total_income'].min()
data['total_income'].max()
data['purpose'].value_counts()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод**

В таблице **12 столбцов** и **21525 строк**. Тип данных во всех столбцах — **object, int64, float64**.

Таблица содержит данные о заемщике в следующем виде:
**children** — количество детей в семье,
**days_employed** — общий трудовой стаж в днях,
**dob_years** — возраст клиента в годах,
**education** — уровень образования клиента,
**education_id** — идентификатор уровня образования,
**family_status** — семейное положение,
**family_status_id** — идентификатор семейного положения,
**gender** — пол клиента,
**income_type** — тип занятости,
**debt** — имел ли задолженность по возврату кредитов,
**total_income** — ежемесячный доход,
**purpose** — цель получения кредита,

В названиях колонок нет нарушений, однако, количество значений в столбцах различается. Значит, в данных есть пропущенные значения. В столбце education встречаются шрифты разного регистра, в столбце days_employed присутсвуют значения со знаком '-' и аномально большие значения. В столбце gender выявлено одно значение XNA, возможно произошла ошибка при выгрузке или заполнении.  Для корректного анализа эти данные необходимо обработать.

В каждой строке находятся данные о заемщике. Часть колонок содержит общую информацию (возраст, пол, образование, количесво детей, тип занятости, семейное положение). Другие колонки напряму связаны с его банковской историей (наличие/отсутствие задодженности, ежемесячный доход, цель получения кредита). 

Можно сказать, что данных достаточно для анализа, однако, необходимо устранить пропуски.



## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
# используем метод isna(), чтобы найти пропущенные значения в столбцах, где данные, очевидно, пропущенны. Посчитали их припомощи метода len()
len(data[data['days_employed'].isna()]) # 2174
len(data[data['total_income'].isna()]) # 2174

# работа над заменой пропусков в days_employed и total_income
data.loc[data['days_employed'].isna(), 'days_employed'] = data['days_employed'].sort_values(ascending=True).median()
data.loc[data['total_income'].isna(), 'total_income'] = data['total_income'].sort_values(ascending=True).median()

# нулевые значения в возрасте заменим на средние
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].sum() / data['dob_years'].count()
data.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


**Вывод**

Первым делом необходимо обнаружить пропуски. Для этого использовали метод isna() и len(), чтоб подсчитать их количество. Получили по 2174 в колонках days_employed и total_income. Так как данные по колическву дней и суммарному доходу представляют значимость для исследования, причем составляют 10% от общих данных, их нельзя удалить вместе со всей строкой. Нужно сказать, что это количественные переменные. Пропуски в таких переменных заполняют характерными значениями. Чтобы примерно оценить типичные значения выборки, в данном случае можно применить среднее арифметическое или медианное значение. 
Нули в графе dob_years заменили на среднее значение.
После проверки видимо, что все пропуски заполнены.
Возможно, подобные ошибки появились при выгрузке данных, либо их причина - человеческий фактор.

### Замена типа данных

In [3]:
# перевод вещественных чисел в целочисленные
data['days_employed'] = data['days_employed'].apply(abs)
data['days_employed'] = data['days_employed'].astype('int')
data.loc[data['days_employed'] > 30000, 'days_employed'] = data['days_employed'] / 24
data['days_employed'] = data['days_employed'].astype('int')

data['total_income'] = data['total_income'].astype('int')

data['dob_years'] = data['dob_years'].astype('int')


**Вывод**

Существует несколько способов перевода из одного типа данных в другой, в том числе to_numeric(), однако он подходит для  перевода строковых значений в вещественные. В данном пункты переводим вещественные числа в целочисленные методом astype('int'). 

### Обработка дубликатов

In [4]:
data.duplicated().sum() # нашли 54 дубликата
 # удалили 54 дубликата

# проверим наличие скрытых дуьликатов в столбце children
data['children'].unique()
#array([ 1,  0,  3,  2, -1,  4, 20,  5])
data['children'].value_counts() 
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
data['children'].value_counts()


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [5]:
data['gender'].value_counts()
# доля аномалий в колонке gender 0,05% и она не участвует в исследовании, поэтому можно ее не трогать


F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [6]:
# проверим наличие скрытых дуьликатов в столбце education
data['education'].unique()
data['education'] = data['education'].str.lower()
data['education'].value_counts()



среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [7]:
# проверим наличие скрытых дуьликатов в столбце family_status
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [8]:
# проверим наличие скрытых дуьликатов в столбце income_type
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [9]:
# проверим наличие скрытых дуьликатов в столбце purpose
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [10]:
# проверим наличие скрытых дуьликатов в столбце debt
data['debt'].unique()

array([0, 1])

In [11]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

**Вывод**

Для поиска дубликатов применили метод duplicated() и произвели их подсчет методом sum(). Таким образом были выявлены строки с полным совпадением данных. Для удаления дубликатов использовали метод data.drop_duplicates().reset_index(drop=True), чтобы не создавать столбец со старыми значениями индексов.

Для поиска неявных дубликатов применили метод unique() к слолбцам education, children, debt, purpose, income_type, family_status, education, gender. Обнаружили повторы из-за разного регистра в колонке education, привели все к нижнему регистру методом str.lower(). При проверке столбца children были обнаружены подозрительные значения -1 и 20. Подобные значения могли возникнуть при выгрузке данных. Они могут обозначать 1 ребенка и 2х соотвественно. Доля этих значений несущественна, поэтому логичнее всего заменить эти значения на 1 и 2 при помощи метода replace().

После проверки указанных столбцов на наличие дубликатов удалили все дубликаты при помощи метода drop_duplicates().


### Лемматизация

In [12]:
# выгружаем библиотеку Pymystem3 для лемматизации
from pymystem3 import Mystem
from collections import Counter # используем для подсчета повторов
m = Mystem() 
text = data['purpose'].unique()
separator = ' '
text = separator.join(text)
lemmas = (m.lemmatize(text))
display(Counter(lemmas)) 

#пишем функцию
def lemmas_purpose(row):
    purpose = row['purpose']
    purpose_lemmas = m.lemmatize(purpose)
    if ('недвижимость' in purpose_lemmas or 'жилье' in purpose_lemmas):
        return 'недвижимость'
    elif ('свадьба' in purpose_lemmas or 'сыграть' in purpose_lemmas):
        return 'свадьба'
    elif 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    elif 'образование' in purpose_lemmas:
        return 'образование'
    else:
        return 'цель не определена'

data['general_purposes'] = data.apply(lemmas_purpose, axis = 1)
display(data.head())
data['general_purposes'].value_counts()
    
 

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,general_purposes
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: general_purposes, dtype: int64

**Вывод**

Находим леммы в значениях столбца с целями получения кредита и осуществляем их подсчет. Это может выявить основные цели получения кредита.
Для лемматизиции необходимо выгрузить бублиотеку Pymystem3, далее нашли все уникальные значения для колонки purpose и провели леммизацию. С помощью Counter посчитали частоту появления слов. Дальше выделили самые часто встречающиеся категории при помощи ввода функции, в которой метод apply лемматизирует каждую ячейку столбца purpose и проверяет какая категория обнаружится.
Таким образом обнаружили 4 самые большие группы заемщиков.

### Категоризация данных

In [13]:
# выделяем категории
data[['education']].drop_duplicates()


,education
0,высшее
1,среднее
13,неоконченное высшее
31,начальное
2962,ученая степень


In [14]:
data[['family_status']].drop_duplicates()


,family_status
0,женат / замужем
4,гражданский брак
18,вдовец / вдова
19,в разводе
24,Не женат / не замужем


In [15]:
data[['family_status_id', 'family_status']].drop_duplicates().set_index('family_status_id')


,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


In [16]:
data[['income_type']].drop_duplicates()


,income_type
0,сотрудник
4,пенсионер
5,компаньон
26,госслужащий
3132,безработный
5931,предприниматель
9399,студент
20778,в декрете


In [17]:
data[['purpose']].drop_duplicates()


,purpose
0,покупка жилья
1,приобретение автомобиля
3,дополнительное образование
4,сыграть свадьбу
6,операции с жильем
7,образование
8,на проведение свадьбы
9,покупка жилья для семьи
10,покупка недвижимости
11,покупка коммерческой недвижимости


**Вывод**

На данном этапе изучили словари с категориальными переменными. Категоризация служит инструментом обнаружения того, какие категории вообще есть в датафрейме и в каком формате они записаны.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [18]:
# Делаем сводную таблицу для children и debt
children_pivot = data.pivot_table(index = ['children'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
children_pivot['mean'] = children_pivot['mean'] * 100 # получили процент
children_pivot.columns = ['has_debt', 'total', '%'] # создали колонки
children_pivot = children_pivot.sort_values(by = 'children', ascending = False) # отсортировали в порядке убывания
display(children_pivot)

,has_debt,total,%
children,,,
5,0,9,0.000000
4,4,41,9.756098
3,27,330,8.181818
2,202,2128,9.492481
1,445,4855,9.165808
0,1063,14091,7.543822


**Вывод**

По итогам сводной таблицы мы видим, что самый большой процент имеющих кредит (9,7%) имеют 4х детей, далее следуют семьи с 2мя детьми (9,5%) и семьи с 1 ребенком (9,1%). Доля семей с 3 детьми в среднем на 1% ниже, чем с одним, двумя и четыремя детьми. Однако доля имеющих кредит среди семей, не имеющих детей, равна 7,5%. Таким образом средний показатель долга для семей с детьми - 9,1%, в то время как только 7,5% семей без детей имеют кредит. Разница составляет полтора процента и не кажется существенной.
Также стоит иметь в виду, что абсолютное количество кредитов в семьях без детей выше.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [19]:
# Делаем сводную таблицу для family_status и debt
family_status_pivot = data.pivot_table(index = ['family_status'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
family_status_pivot['mean'] = family_status_pivot['mean'] * 100 # получили процент
family_status_pivot.columns = ['has_debt', 'total', '%'] # создали колонки
family_status_pivot = family_status_pivot.sort_values(by = 'family_status', ascending = False) # отсортировали в порядке убывания
display(family_status_pivot)

,has_debt,total,%
family_status,,,
женат / замужем,931,12339,7.545182
гражданский брак,388,4151,9.347145
вдовец / вдова,63,959,6.569343
в разводе,85,1195,7.112971
Не женат / не замужем,274,2810,9.750890


**Вывод**

В данном случае мы видим, что больше всего кредитов имеют неженатые/незамужние люди (9,7%) и люди в гражданском браке (9,2%). Женатые/замужние и люди в разводе имеют по 7,5% и 7,1% соответственно. Меньше всего (6,6%) кредитов у вдовцов. Разница между каждой группой, представленной в семейном положении не превышает 2,2%. Из этого можно сделать вывод, что семейное положение практически не влияет на возврат кредита.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [21]:
# Делаем сводную таблицу для income_type и debt

income_type_pivot = data.pivot_table(index = ['income_type'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
income_type_pivot['mean'] = income_type_pivot['mean'] * 100 # получили процент
income_type_pivot.columns = ['has_debt', 'total', '%'] # создали колонки
income_type_pivot = income_type_pivot.sort_values(by = 'income_type', ascending = False) # отсортировали в порядке убывания
display(income_type_pivot)

,has_debt,total,%
income_type,,,
студент,0,1,0.000000
сотрудник,1061,11084,9.572357
предприниматель,0,2,0.000000
пенсионер,216,3829,5.641160
компаньон,376,5078,7.404490
госслужащий,86,1457,5.902539
в декрете,1,1,100.000000
безработный,1,2,50.000000


**Вывод**

В данной таблице наблюдаем тот факт, что больше всего кредитов у сотрудников компаний (9,5%). На втором месте  компаньоны - 7,4%, госслужащие и пенсионеры имеют самый низкий процент кредитов - 5,8% и 5,6%. В абсолютном выражении люди в декрете и безработные не представляют статистической значимости. По данной группе также не выявлено значительных колебаний в возврате долга, что приводит к выводу, что зависимость  между доходом и возвратом долга отсутствует. 

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# Делаем сводную таблицу для general_purposes и debt
pivot_purposes = data.pivot_table(index = ['general_purposes'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
pivot_purposes['mean'] = pivot_purposes['mean'] * 100
pivot_purposes.columns = ['has_debt', 'total', '%']
pivot_purposes = pivot_purposes.sort_values(by = '%', ascending = False)
display(pivot_purposes)

**Вывод**

Цели кредита, очевидно также существенно не влияют на его возврат. В данной таблице мы видим колебания от 7,2% до 9,3% по категориям.

## Шаг 4. Общий вывод

В данном проекте были проанализированы даные статистики о платёжеспособности клиентов банка с целью выяления зависимости между семейным положением и количеством детей клиента на факт погашения кредита в срок. 

Перед началом анализа данные были выгружены из папки и датафрейм был проанализирован при помощи методов info() и value_counts(). Это помогло ознакомиться с размером датафрейма и  типами данных в нем и увидеть потенциальные проблемы. Таким образом обнаружилось, что в данных присутствовали пропуски, дубликаты и аномальные значения. Также нужно сказать, что в таблице были представлены данные вида object, int64, float64. 

На следующем шаге происходила поэтапная предобработка данных.

Во-первых, была произведена обработка пропусков с использованием метода isna() в столбцах days_employed,
total_income, dob_years.

Во-вторых, привели типы данных в формат int методом astype. Для столбца days_employеd предварительно произвели замену  вещественных чисел на целочисленные, убрали знак "-" и перевели данные в часы.

Далее был произведен поиск явных и неявных дубликатов при помощи  метода unique() к столбцам education, children, debt, purpose, income_type, family_status, education, gender. Обнаружили повторы из-за разного регистра в колонке education, привели все к нижнему регистру методом str.lower(). При проверке столбца children были обнаружены подозрительные значения -1 и 20. Подобные значения могли возникнуть при выгрузке данных. Они могут обозначать 1 ребенка и 2х соответственно. Доля этих значений несущественна, поэтому заменили эти значения на 1 и 2 при помощи метода replace(). Остальные данные не участвуют в анализе или их доля несущественна.

На следующем этапе предобработки произвели лемматизацию столбца purpose с целью обозначения общих категорий целей кредита. Для этого написали функцию и сохранили результат в отдельном столбце general_purposes.

Далее была произведена категоризация данных. Для этого выделили словари с категориальными переменными, которые необходимы для решения задачи проекта по поиску зависимости между семейным положением и количеством детей клиента на факт погашения кредита в срок.

В пункте три анализа было составлено 4 таблицы:
1. Сводная таблица для children и debt. В ней был произведен анализ с целью ответа на вопрос, есть ли зависимость между количеством детей и возвратом кредита в срок. По итогам сводной таблицы мы видим, что самый большой процент имеющих кредит (9,7%) имеют 4х детей, далее следуют семьи с 2мя детьми (9,5%) и семьи с 1 ребенком (9,1%). Доля семей с 3 детьми в среднем на 1% ниже, чем с одним, двумя и четыремя детьми. Однако доля имеющих кредит среди семей, не имеющих детей, равна 7,5%. Таким образом средний показатель долга для семей с детьми - 9,1%, в то время как только 7,5% семей без детей имеют кредит. Разница составляет полтора процента и не кажется существенной.

2. Сводная таблица для family_status и debt. В ней был произведен анализ с целью ответа на вопрос, есть ли зависимость между семейным положением и возвратом кредита в срок. Из таблицы очевидно, что больше всего кредитов имеют неженатые/незамужние люди (9,7%) и люди в гражданском браке (9,2%). Женатые/замужние и люди в разводе имеют по 7,5% и 7,1% соответственно. Меньше всего (6,6%) кредитов у вдовцов. Разница между каждой группой, представленной в семейном положении не превышает 2,2%. Из этого можно сделать вывод, что семейное положение практически не влияет на возврат кредита.

3. Сводная таблица для income_type и debt. В ней был произведен анализ с целью ответа на вопрос, есть ли зависимость между доходом и возвратом кредита в срок. В данной таблице наблюдаем тот факт, что больше всего кредитов у сотрудников компаний (9,5%). На втором месте компаньоны - 7,4%, госслужащие и пенсионеры имеют самый низкий процент кредитов - 5,8% и 5,6%. В абсолютном выражении люди в декрете и безработные не представляют статистической значимости. По данной группе также не выявлено значительных колебаний в возврате долга, что приводит к выводу, что зависимость между доходом и возвратом долга отсутствует.

4. Сводная таблица для general_purposes и debt. В ней был произведен анализ с целью ответа на вопрос, есть ли зависимость между доходом и возвратом кредита в срок. Цели кредита, очевидно также существенно не влияют на его возврат. В данной таблице мы видим колебания от 7,2% до 9,3% по категориям.

По итогам исследования можно сделать вывод, что наличие детей и семейное положение не влияет на возврат кредита в срок. Анализ каждой категории показал, что каждая группа закредитована в промежутке от 6,6% до 9,7%. 
Также из данных очевидно, что доход и цель кредита также не влияют на возвратность кредита в срок.

Исходя из анализа данных, можно сделать вывод, что заказчику в первую очередь стоит дополнить данные для дальнейшего анализа и продолжить поиск потенциальных категорий, которые могли оказать более существенное влияние для возврат кредита в срок.
Тем не менее, не стоит забывать, о том, что данные пополняются, и, возможно, при изменении кредитной политики банка, эти данные покажут новые виды зависимости, которые будут полезны в будущем. 




## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.